Import Libraries

In [10]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App

-  create app.py file to make flask app
-  create Flask app to execute all of the endpoint you have made

2. 2 points - Created functionality to read or get specific data from the database

- create query to read data from database
- create function to execute read specific information into table from database

3. 4 points - Created functionality to input new data into each table for the databases

- create query to insert new data into stations and trips table
- create function to execute input data into stations and trips table

4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)

- create query to make analytical resultfrom the data
- create static endpoint to analyze the data from database, for example average trip durations
    
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)

- create query to make analytical resultfrom the data
- create dinamic endpoint to analyze the data from database, for example average trip durations for each bike_id

6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

- create input data for refering into query for post endpoint
- make query and aggregation function to implement the input

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [2]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [3]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16


**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [4]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [11]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()

In [6]:
data = tuple(stations.iloc[1].fillna('').values)

data

(2500,
 'Republic Square',
 'closed',
 '425 W 4th Street',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 9,
 '2021-01-04T12:00:00Z')

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [7]:
# Get the data values
data = tuple(stations.iloc[1].fillna('').values)

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""

In [8]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (2500, 'Republic Square', 'closed', '425 W 4th Street', '', '', '', '', '', '', '', '', 9, '2021-01-04T12:00:00Z')



In [9]:
# Execute the query 
conn.execute(query)

IntegrityError: UNIQUE constraint failed: stations.station_id

In [ ]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [ ]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [ ]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[6].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [ ]:
result

'Error'

**TASK: Create a Function to insert data for `trips` table**

In [ ]:
# Your code here
def insert_into_trips(data, conn):
    # Make the query
    query = f"""
    INSERT INTO trips
    VALUES {data}
    """

    try:
        conn.execute(query)
    except:
        return "Error"
    
    conn.commit()
    return "Ok"


In [ ]:
conn = make_connection()
data = tuple(trips.iloc[2].fillna('').values)

insert_into_trips(data, conn)

'Error'

## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [ ]:
# Get Specific Station ID  Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [ ]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2500



In [ ]:
# Run the query and get the result
result = pd.read_sql_query(query_station_all, conn)

In [ ]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
1,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [ ]:
def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

**TASK: Create a Function to read data from `trips` table**

In [ ]:
# Your code here
def get_trip_id(trip_id, conn):
    query = f"""
            SELECT *
            FROM 
                trips
            WHERE
                trip_id = {trip_id}
            """
    return pd.read_sql_query(query, conn)
    
def get_all_trips(conn):
    query = f"""
            SELECT *
            FROM 
                trips
            """
    return pd.read_sql_query(query, conn)


# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [12]:
# your code
# @app.route('/trips/')
def route_all_trips():
    return get_all_trips().to_json()

def get_all_trips():
    conn = make_connection()
    query = f"""SELECT * FROM trips"""
    return pd.read_sql_query(query, conn)


### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [16]:
url = 'http://127.0.0.1:5000/stations/'
res = requests.get(url)

In [17]:
res

<Response [200]>

In [18]:
res.text

'{"station_id":{"0":2500,"1":2546,"2":2502,"3":3687,"4":3684,"5":3464,"6":2576,"7":2712,"8":3381,"9":1001,"10":1002,"11":1003,"12":1004,"13":1005,"14":1006,"15":1007,"16":1008,"17":2707,"18":4699,"19":2570,"20":2496,"21":3294,"22":2497,"23":4047,"24":2503,"25":2711,"26":2565,"27":3619,"28":3798,"29":3799,"30":3291,"31":2539,"32":2571,"33":2542,"34":2549,"35":2561,"36":3455,"37":3513,"38":3790,"39":4060,"40":2567,"41":2498,"42":2566,"43":2499,"44":3621,"45":3390,"46":2540,"47":2547,"48":2562,"49":2552,"50":4879,"51":3635,"52":3685,"53":3793,"54":4050,"55":4051,"56":4057,"57":2548,"58":2494,"59":2495,"60":2501,"61":2563,"62":3293,"63":4052,"64":4054,"65":4055,"66":4058,"67":4059,"68":4061,"69":4062,"70":3292,"71":2544,"72":2568,"73":2569,"74":3660,"75":2822,"76":2823,"77":2504,"78":2537,"79":3686,"80":3791,"81":3792,"82":3838,"83":3841,"84":4048,"85":3377,"86":3794,"87":3795,"88":3797},"name":{"0":"Republic Square","1":"ACC - West & 12th Street","2":"Barton Springs & Riverside","3":"Boar

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [19]:
pd.DataFrame(res.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
1,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z
2,2502,Barton Springs & Riverside,closed,400 Barton Springs Rd.,,,,,,,,moved to Pedernales/2nd. Original city ID: 16729,9,2021-01-04T12:00:00Z
3,3687,Boardwalk West,active,300 E. Riverside Dr.,,16683,parkland,9,solar,30.0,5.0,,9,2021-01-04T12:00:00Z
4,3684,Congress & Cesar Chavez,active,124 W Cesar Chavez St,,32594,parkland,11,solar,35.0,5.0,parkland at ROW/easement,9,2021-01-04T12:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,4048,South Congress @ Bouldin Creek,active,799 South Congress Avenue,,32746,undetermined_parking,13,solar,40.0,5.0,,9,2021-01-04T12:00:00Z
85,3377,MoPac Pedestrian Bridge @ Veterans Drive,active,2105 Veterans Dr.,,16598,undetermined_parking,19,solar,55.0,5.0,parkland in ROW/eastment,9,2021-01-04T12:00:00Z
86,3794,Dean Keeton & Speedway,active,180 E. Dean Keeton St.,,32670,undetermined_parking,19,solar,55.0,5.0,,9,2021-01-04T12:00:00Z
87,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,,32671,undetermined_parking,19,solar,55.0,5.0,,9,2021-01-04T12:00:00Z


In [20]:
url = 'http://127.0.0.1:5000/trips/'
res = requests.get(url)

In [21]:
res

<Response [200]>

In [22]:
res.text[1:1000]

'"id":{"0":9900289692,"1":9900285987,"2":9900285989,"3":9900285991,"4":9900286140,"5":9900286143,"6":9900286171,"7":9900286214,"8":9900286540,"9":13575843,"10":9900287291,"11":9900287293,"12":9900287294,"13":9900287414,"14":9900287861,"15":9900287864,"16":9900288008,"17":9900288149,"18":9900290433,"19":9900288210,"20":9900288274,"21":9900288276,"22":9900288278,"23":9900288279,"24":9900288333,"25":9900288343,"26":9900288379,"27":9900288445,"28":9900288446,"29":9900288447,"30":9900288594,"31":9900288695,"32":9900288777,"33":9900288832,"34":9900288863,"35":9900288951,"36":9900288985,"37":9900288998,"38":9900289083,"39":9900289091,"40":9900289108,"41":9900289112,"42":9900289126,"43":9900289130,"44":9900289157,"45":9900289236,"46":9900289247,"47":9900289262,"48":9900289307,"49":9900289320,"50":9900289332,"51":9900289349,"52":9900289372,"53":9900289380,"54":9900289457,"55":9900289493,"56":9900289496,"57":9900289535,"58":9900289572,"59":9900289573,"60":9900289581,"61":9900289594,"62":99002896

In [23]:
pd.DataFrame(res.json()).head(10)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
5,9900286143,24-Hour Kiosk (Austin B-cycle),924,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,17
6,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00 UTC,2536,Waller & 6th St.,2536,Waller & 6th St.,6
7,9900286214,Annual Membership (Austin B-cycle),24,2014-10-26 20:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,0
8,9900286540,24-Hour Kiosk (Austin B-cycle),117,2014-10-27 15:12:00 UTC,2536,Waller & 6th St.,2536,Waller & 6th St.,12
9,13575843,Walk Up,302,2017-01-29 16:42:52 UTC,3464,Pease Park,3464,Pease Park,47


## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [24]:
station_id = 2500
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [25]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

In [26]:
trip_id = 8269930
url = f"http://127.0.0.1:5000/trips/{trip_id}"

In [27]:
response = requests.get(url)
pd.DataFrame(response.json())

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,8269930,Local365,460,2016-01-01 00:18:49 UTC,1007,Lavaca & 6th,2501,5th & Bowie,4


## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [28]:
data = {
    "name" : "Andi", 
    "age" : 24,
    "address" : "my adress"
}

url = "http://127.0.0.1:5000/json"

In [29]:
res = requests.post(url, json=data)

In [30]:
res

<Response [200]>

In [31]:
res.text

'Hello Andi, your age is 24, and your address in my adress'

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True)))
    data = tuple(data.fillna('').values)
    
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [32]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [33]:
data = stations.iloc[22].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)

In [35]:
url = "http://127.0.0.1:5000/stations/add"
res = requests.post(url, json=data_json)

In [36]:
res

<Response [200]>

In [37]:
res.text

'Error'

In [38]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table
for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

0it [00:00, ?it/s]

98it [08:36,  5.27s/it]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [67]:
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, json=data_json)

17982it [01:19, 226.18it/s]


In [39]:
res

<Response [200]>

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

In [43]:
# Your Code Here 
# @app.route('/trips/average_duration')
def route_trip_avg_duration():
    return get_trip_avg_duration().to_json()

def get_trip_avg_duration():
    conn = make_connection()
    query = '''
            SELECT
                AVG(duration_minutes) AS average_duration
            FROM
                trips
            '''
    return pd.read_sql_query(query, conn)
    

## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

In [44]:
# Your Code Here
# @app.route('/trips/average_duration/<bike_id>')
def route_trip_avg_duration(bike_id):
    return get_trip_avg_duration_by_id(bike_id).to_json()

def get_trip_avg_duration_by_id(bike_id):
    conn = make_connection()
    query = f'''
            SELECT
                AVG(duration_minutes) AS average_duration
            FROM
                trips
            WHERE
                bikeid = {bike_id}
            '''
    return pd.read_sql_query(query, conn)

## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

After created the endpoint, make sure to implement it to app.py file.

In [45]:
# @app.route('/station-bike-activity_by_period', methods=['POST'])
def route_station_bike_activity_by_period():
    input_data = request.get_json() # Get the input as dictionary
    specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

    get_station_bike_activity_by_period(specified_date).to_json()

def get_station_bike_activity_by_period(specified_date):
    # Subset the data with query 
    conn = make_connection()
    query = f"""
            SELECT *
            FROM
                trips
            WHERE
                start_time LIKE ("{specified_date}%")
            """
    selected_data = pd.read_sql_query(query, conn)

    # Make the aggregate
    result = selected_data.groupby('start_station_id').agg({
        'bikeid' : 'count', 
        'duration_minutes' : 'mean'
    })
    
    # Return the result
    return result

In [46]:
# Your Code Here
data = { "period" : "2015-08" }
url = 'http://127.0.0.1:5000/station-bike-activity_by_period'
res = requests.post(url, json=data)

In [47]:
res

<Response [200]>

In [48]:
res.text

'{"bikeid":{"1007":91,"1008":311,"2494":791,"2495":590,"2496":250,"2497":440,"2498":427,"2499":635,"2501":694,"2502":422,"2503":292,"2504":213,"2536":106,"2537":290,"2538":229,"2539":394,"2540":173,"2541":84,"2542":147,"2544":122,"2545":46,"2547":119,"2548":199,"2549":297,"2550":296,"2552":430,"2561":128,"2562":193,"2563":260,"2564":308,"2565":249,"2566":503,"2567":306,"2568":112,"2569":210,"2570":321,"2571":150,"2572":314,"2575":643,"2576":226,"2707":655,"2711":501,"2712":241,"2822":189,"2823":58,"3291":50,"3292":71,"3293":40,"":707},"duration_minutes":{"1007":18.2307692308,"1008":13.2379421222,"2494":26.0341340076,"2495":15.4593220339,"2496":27.624,"2497":27.4136363636,"2498":22.018735363,"2499":18.0960629921,"2501":15.476945245,"2502":31.2819905213,"2503":29.0719178082,"2504":21.9530516432,"2536":32.1509433962,"2537":12.1206896552,"2538":25.3886462882,"2539":22.9035532995,"2540":26.9190751445,"2541":22.75,"2542":14.5238095238,"2544":14.8442622951,"2545":11.2608695652,"2547":52.48739

In [49]:
pd.DataFrame(res.json())

,bikeid,duration_minutes
1007,91,18.230769
1008,311,13.237942
2494,791,26.034134
2495,590,15.459322
2496,250,27.624000
2497,440,27.413636
2498,427,22.018735
2499,635,18.096063
2501,694,15.476945
2502,422,31.281991


# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. Make sure you have implemented all the endpoints to `app.py` (including your custom static, dynamic and post endpoints).
3. Submit your `Bikeshare API.ipynb` and `app.py` file in your github repository. The dataset is optional to post since it has big size to post. 

